In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/cev-eurovolley-womens-player-stats/all_data.csv', header=[0, 1], index_col=0)
df.head(10)

# Setters

In [ ]:
setters = df[df['Role']['Unnamed: 1_level_1'] == 'Setter']
setters.head(5)

In [ ]:
# get the setters played at least 20 sets
setters = setters[setters['Played']['Sets'] >= 20]
setters

In [ ]:
setters['Attack', 'Points Per Set'] = setters['Attack', 'Total Points'] / setters['Played', 'Sets']

In [ ]:
# these have the highest attack points per set!
setters['Attack'].sort_values(by='Points Per Set', ascending=False).head(5)

In [ ]:
# these have the highest block points per set!
setters['Block'].sort_values(by='Points per Set', ascending=False).head(5)

In [ ]:
# get these setters names
top_attackers = setters['Attack'].sort_values(by='Points Per Set', ascending=False).head(5).index

In [ ]:
setters.loc[top_attackers]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# lets see if theres a relationship between ace per set and attack points per set
# in order to do this first plot these 2 column as graph
ace_per_set = setters['Serve', 'Ace per Set']
attack_points_per_set = setters['Attack', 'Points Per Set']

fig , ax1 = plt.subplots(figsize=(12, 6))

ax2 = ax1.twinx()
ax1.bar(ace_per_set.index, ace_per_set.values)
ax2.plot(attack_points_per_set.index, attack_points_per_set.values, color='red')

ax1.set_xlabel('Player Name')
ax1.set_ylabel('Ace Per Set', color='blue')
ax2.set_ylabel('Attack Points Per Set', color='red')
ax1.set_xticklabels(ace_per_set.index, rotation='vertical')
fig.show()


### so it seems we can't understand anything from this

In [ ]:
# lets try to see correlation
np.corrcoef(ace_per_set.values, attack_points_per_set.values)[0][1]

### it seems theres not that much a correlation between being agressive in attack and serving

In [ ]:
block_points_per_set = setters['Block', 'Points per Set']
attack_points_per_set = setters['Attack', 'Points Per Set']

fig , ax1 = plt.subplots(figsize=(12, 6))

ax2 = ax1.twinx()
ax1.bar(block_points_per_set.index, block_points_per_set.values)
ax2.plot(attack_points_per_set.index, attack_points_per_set.values, color='red')

ax1.set_xlabel('Player Name')
ax1.set_ylabel('block Points Per Set', color='blue')
ax2.set_ylabel('Attack Points Per Set', color='red')
ax1.set_xticklabels(block_points_per_set.index, rotation='vertical')
fig.show()

In [ ]:
np.corrcoef(block_points_per_set.values, attack_points_per_set.values)[0][1]

### so it seems there's a correlation between attack points and block points

In [ ]:
# lets see all the correlations
setters[['Serve', 'Attack', 'Block']].corr()

# Outside Hitters

In [ ]:
outsides = df[df['Role', 'Unnamed: 1_level_1'] == 'Outside Hitter']
outsides.head(5)

In [ ]:
outsides = outsides[outsides['Played', 'Sets'] >= 20]
outsides.shape[0] # there are 55 players who played at least 20 sets 

In [ ]:
def convert_to_percentage(str_percentage):
    return float(str_percentage.split('%')[0]) / 100

In [ ]:
outsides['Attack', 'Excellent %'] = outsides['Attack', 'Excellent %'].apply(convert_to_percentage)

In [ ]:
outsides['Attack'].sort_values(by='Excellent %', ascending=False)

In [ ]:
outsides['Attack', 'Points per Set'] = outsides['Attack', 'Total Points'] / outsides['Played', 'Sets']
outsides['Attack'].sort_values(by='Excellent %', ascending=False)

In [ ]:
outsides['Attack', 'Lost Points'] = outsides['Attack', 'Error'] + outsides['Attack', 'Blocked']
outsides['Attack']

In [ ]:
outsides['Attack', 'Lost Points / Total'] = outsides['Attack', 'Lost Points'] / outsides['Attack', 'Total Points']
outsides['Attack']

In [ ]:
outsides['Attack'].sort_values(by='Lost Points / Total')

In [ ]:
outsides['Attack'].corr()

In [ ]:
# drop NaN
outsides['Reception'].dropna(inplace=True)

In [ ]:
outsides['Reception', 'Error/Total'] = outsides['Reception', 'Error'] / outsides['Reception', 'Total']
outsides['Reception']

In [ ]:
reception = outsides['Reception'].dropna()
reception

In [ ]:
reception = reception[reception['Total'] > 100] 
reception.sort_values(by='Error/Total')

In [ ]:
reception['Excellent %'] = reception['Excellent %'].apply(convert_to_percentage)

In [ ]:
reception['Efficiency'] = reception['Efficiency'].apply(convert_to_percentage)

In [ ]:
reception.sort_values(by='Excellent %', ascending=False)

In [ ]:
reception['Negative/Total'] = reception['Negative'] / reception['Total']

In [ ]:
reception['My Efficiency'] = reception['Excellent %'] / (1.2 * reception['Error/Total'] + 0.4 * reception['Negative/Total'])
reception.sort_values(by='My Efficiency', ascending=False)

In [ ]:
reception.corr()

In [ ]:
outsides['Attack', 'Efficiency'] = outsides['Attack', 'Efficiency'].apply(convert_to_percentage)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
outsides['Attack', 'My Efficiency'] = (100 * outsides['Attack', 'Excellent %'] + outsides['Attack', 'Points per Set']) / (0.25 + outsides['Attack', 'Lost Points / Total']) / 50

outsides['Attack'].sort_values(by='My Efficiency', ascending=False)

In [ ]:
outsides['Attack'].corr()

In [ ]:
outsides['Attack'].head()

In [ ]:
outsides['Block'].sort_values(by='Points per Set', ascending=False).head(10)
# theres not much of a information in Block section.
# numbers are too small to get a meaning from them

In [ ]:
outsides['Reception', 'My Efficiency'] = 0
outsides['Reception'] = reception
outsides['Reception']

### our outside hitters data includes some opposite players.
### they are actually playing opposite, opposites usually don't receive a ball, their [Reception, Error/Total] values are NaN
### so we will drop NaN values.
### we will drop these players

In [ ]:
outsides.dropna(inplace=True)
outsides

### we have 42 players now

In [ ]:
dfx = outsides[['Attack', 'Reception']] # we only need these columns now
dfx.head()

In [ ]:
dfx['Reception'].head()

In [ ]:
dfx['Attack'].head()

### does the player with good attacks have good defense too?

In [ ]:
np.corrcoef(dfx['Attack', 'My Efficiency'], dfx['Reception', 'My Efficiency'])[0][1]
# little bit

### lets find the best players in offense and defense combined

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# in order to do this we can use 2 My Efficiency values that we created for each Attack and Reception
attack_efficiency = dfx['Attack', 'My Efficiency']
reception_efficiency = dfx['Reception', 'My Efficiency']

fig , ax1 = plt.subplots(figsize=(12, 6))

ax2 = ax1.twinx()
ax1.bar(attack_efficiency.index, attack_efficiency.values)
ax2.plot(reception_efficiency.index, reception_efficiency.values, color='red')

ax1.set_xlabel('Player Name')
ax1.set_ylabel('Ace Per Set', color='blue')
ax2.set_ylabel('Attack Points Per Set', color='red')
ax1.set_xticklabels(attack_efficiency.index, rotation='vertical')
fig.show()

### lets scale the 'my efficiency'columns to (0,1)

In [ ]:
dfx['Attack', 'My Efficiency']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfx['Attack', 'My Efficiency'] = scaler.fit_transform(dfx['Attack', 'My Efficiency'].values.reshape(-1, 1))
dfx['Reception', 'My Efficiency'] = scaler.fit_transform(dfx['Reception', 'My Efficiency'].values.reshape(-1, 1))
dfx.head()

In [ ]:
# Best attack players
dfx['Attack'].sort_values(by='My Efficiency', ascending=False).head(10)

In [ ]:
# Best defense players
dfx['Reception'].sort_values(by='My Efficiency', ascending=False).head(10)

### Lets say we calculate the general goodness of a player with this 3 different formulas:

### player_score = 2 * attack_score + reception_score for attack first consideration
#### We consider attack is 2 times important than defense

### player_score = attack_score + 2 * reception_score for defense first consideration
#### We consider defense is 2 times important than attack

### player_score = 1.5 * attack_score + 1.5 * reception_score for balanced consideration
#### Attack and defense is equally important on this one

### NOTE: score means 'My Efficiency' column

In [ ]:
dfx['Score', 'With Attack Priority'] = 2 * dfx['Attack', 'My Efficiency'] + dfx['Reception', 'My Efficiency']
dfx['Score', 'With Defense Priority'] = dfx['Attack', 'My Efficiency'] + 2 * dfx['Reception', 'My Efficiency']
dfx['Score', 'With No Priority'] = 1.5 * dfx['Attack', 'My Efficiency'] + 1.5 * dfx['Reception', 'My Efficiency']

In [ ]:
dfx.sort_values(by=('Score', 'With Attack Priority'), ascending=False).head()

In [ ]:
dfx.sort_values(by=('Score', 'With Defense Priority'), ascending=False).head()

In [ ]:
dfx.sort_values(by=('Score', 'With No Priority'), ascending=False).head()

#### These players are the bests according to data.
#### Elena PIETRINI is top 3 in all of them and first at 2 of them, I guess she was the best in this tournament :)
#### Shes Outside Hitter for Italian National Team and Italy is one of the best teams in Women's Volleyball, so it makes sense.

## Now it is time to analyze some Middle Blockers!